# Application of amplitude estimation to option pricing

One of the tasks in finance is that of computing the price of a derivatives contract. Usually, the problem of computing the price of a derivatives contract can be reduced to computing an expectation. In the first part of this notebook we will use the amplitude estimation algorithm to compute an expectation given an input function $f(x)$ and a proability density $p(x)$.

$$\mathbb{E}[f]=\int_a^bp(x)f(x)dx$$

This integral can be approximated by the Riemann sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx$$

In the second part of this notebook we will use this same strategy to price a call option.

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Defining the problem

As we cannot work with a continuous variable we will define a discrete probability distribution and a the value of a function $f$ in the same points where the probability distribution is defined. Here $N = 2^n$ is the size of the discretized probability distribution and the size of the array. In this specific example $n = 3$ and $N = 8$.

In [ ]:
n = 3
N = 2**n
x = np.arange(N)

The discrete probability distribution has to be properly normalised:

$$p_d = \dfrac{1}{0+1+2+3+4+5+6+7}\left(0,1,2,3,4,5,6,7\right),$$
which is saved in the variable *probability*.

p_X = x/np.sum(x)

The discretized function will be:

$$f = \dfrac{1}{7}\left(0,1,2,3,4,5,6,7\right).$$

Note that it is properly normalised



In [ ]:
f_X = x/np.max(x)

## 2. Standard procedure

The most widely approach used in the literature to compute:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx$$

begins defining an initial $n+1$ qbits state:

$$|0\rangle \otimes|0\rangle_{n}$$

In rightmost register we will load the discrete probability distribution:

$$\mathbb{1}\otimes\mathcal{P}|0\rangle \otimes|0\rangle_{n} = |0\rangle\otimes\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle+\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right]$$

Then we load the square root of the function controlled to the leftmost register:

$$|\Psi\rangle=\mathcal{F}\left(I\otimes\mathcal{P}\right)|0\rangle\otimes|0\rangle_{n} = |0\rangle\otimes\left[\sqrt{p_0f_0}|0\rangle+\sqrt{p_1f_1}|1\rangle+\sqrt{p_2f_2}|2\rangle+\sqrt{p_3f_3}|3\rangle+\sqrt{p_4f_4}|4\rangle+\sqrt{p_5f_5}|5\rangle+\sqrt{p_6f_6}|6\rangle+\sqrt{p_7f_7}|7\rangle\right]+...$$

For doing this operations *load_probability* and *load_array* functions from **QQuantLib/DL/data_loading** module will be used (see notebook **01_DataLoading_Module_Use.ipynb**)

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array, uniform_distribution

In [ ]:
p_gate = load_probability(p_X)
f_gate = load_array(np.sqrt(f_X))

Now we compound the two operators $\mathcal{P}$ aqnd $\mathcal{F}$ for creating the necesary loading circuit

In [ ]:
oracle1 = qlm.QRoutine()
register1 = oracle1.new_wires(f_gate.arity)
oracle1.apply(p_gate, register1[:p_gate.arity])
oracle1.apply(f_gate, register1)

In [ ]:
%qatdisplay oracle1 --depth 0 --svg

Now that the data is loaded into the quantum circuit we will see how can we retrieve:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx$$

We express the quantum state resulting from the loading proccess: $|\Psi\rangle$ as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle=\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$
\begin{array}{l}
&\sqrt{a}|\Psi_{0}\rangle = |0\rangle \otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n},\\\\
&\sqrt{1-a}|\Psi_{1}\rangle = |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}.
\end{array}
$$
The probability of measuring $|0\rangle$ in the leftmost qubit is:

$$ a = \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2,$$

so estimating $a$ gives as an approximation of the value of the integral.

For doing this measurement as fast as possible we will use *run* function from **QQuantLib/AE/iterative_quantum_ae.py**. For more information on this topic check the corresponding notebook.

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

In [ ]:
target = [0]
index = [3]
iqae = IQAE(oracle = oracle1,target = target, index = index)

In [ ]:
epsilon = 0.001
N = 1000
alpha = 0.05 
bounds = iqae.run(epsilon = epsilon,N = N,alpha = alpha)

In [ ]:
print("Estimation: ",bounds)
print("Exact solution: ",np.dot(p_X,f_X))

## 3. Problems of the standard procedure

Note that standard way of computing this sum has one major problem. If we have negative values in $f$, we won't be able to compute the correct sum because, when we measure the leftmost qubit we are taking the absolute value:
$$\sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2 = \sum_{i=0}^{2^{n}-1}p(x_i)\left|f(x_i)\right| $$
To demonstrate it let us define a new function $f$ which is negative int the first half:
$$f = \dfrac{1}{7}\left(-0,-1,-2,-3,4,5,6,7\right).$$

In [ ]:
f2 = np.copy(f_X)
f2[1] = -f_X[1]
f2[2] = -f_X[2]
f2[3] = -f_X[3]
print(f2)

We generate the oracle:

In [ ]:
f2_gate = load_array(np.sqrt(np.abs(f2)))
oracle2 = qlm.QRoutine()
register2 = oracle2.new_wires(n+1)
oracle2.apply(p_gate, register2[:n])
oracle2.apply(f2_gate, register2)

We solve it as before.

In [ ]:
target = [0]
index = [3]
iqae = IQAE(oracle = oracle2,target = target, index = index)

In [ ]:
epsilon = 0.001
N = 100
alpha = 0.05 
bounds = iqae.run(epsilon = epsilon,N = N,alpha = alpha)

In [ ]:
print("Estimation: ",bounds)
print("Exact solution: ",np.dot(p_X,f2))

## 4. First proposal of solution

To avoid this problem there is an easy workaround. Instead of loading the discrete probability distribution as a probability, we will load it as a function. This requires an extra qubit. The base probability distribution will now be a uniform distribution. As a summary we define the state:
$$|\Psi\rangle = \dfrac{1}{\sqrt{N}}|0\rangle|0\rangle\left[f_0 p_0|0\rangle+f_1 p_1|1\rangle+f_2 p_2|2\rangle+f_3 p_3|3\rangle+f_4 p_4|4\rangle+f_5 p_5|5\rangle+f_6 p_6|6\rangle+f_7 p_7|7\rangle\right]+...$$

In [ ]:
p3_gate = load_array(p_X)
f3_gate = load_array(f2,id_name = "2")

To be able to compute:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx,$$

we do a Hadamard transform of the state $|\Psi\rangle$:
$$\mathbb{1}\otimes\mathbb{1}\otimes H|\Psi\rangle = \dfrac{1}{N}|0\rangle|0\rangle\left[f_0 p_0+f_1 p_1+f_2 p_2+f_3 p_3+f_4 p_4+f_5 p_5+f_6 p_6+f_7 p_7\right]|0\rangle+...$$



In [ ]:
oracle3 = qlm.QRoutine()
register3 = oracle3.new_wires(n+2)
oracle3.apply(uniform_distribution(n),register3[:n])
oracle3.apply(p3_gate, [register3[:n],register3[n]])
oracle3.apply(f3_gate, [register3[:n],register3[n+1]])
oracle3.apply(uniform_distribution(n),register3[:n])

In [ ]:
%qatdisplay oracle3 --depth 0 --svg

Last, we express the quantum state resulting from the loading proccess as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$
\begin{array}{l}
&\sqrt{a}|\Psi_{0}\rangle = \left[\dfrac{1}{N}\sum_{i=0}^{2^{n}-1}p(x_i)f(x_i)\right]|0\rangle \otimes|0\rangle \otimes|0\rangle,\\\\
\end{array}
$$
The probability of measuring $|0\rangle \otimes|0\rangle \otimes|0\rangle$ is:

$$ a= \left|\dfrac{1}{N}\sum_{i=0}^{2^{n}-1}p(x_i)f(x_i)\right|^2,$$

so then computing $\sqrt{a}$ gives us an estimation.

In [ ]:
target = [0,0,0,0,0]
index = [0,1,2,3,4]
iqae = IQAE(oracle = oracle3,target = target, index = index)

In [ ]:
epsilon = 0.001
N = 100
alpha = 0.05 
bounds = iqae.run(epsilon = epsilon,N = N,alpha = alpha)

In [ ]:
print("Estimation: ",2**n*np.sqrt(bounds[0]),2**n*np.sqrt(bounds[1]))
print("Exact solution: ",np.dot(p_X,f2))

## 5. Problems of the first proposal of solution

We have solved one of the issue, but we still face one major problem. If the expectation of $f$ is negative, we won't be able to notice it, again because we are taking the absolute value.
To demonstrate it let us define a new linear function $f$ which is negative in the second half:
$$f = \dfrac{1}{7}\left(0,1,2,3,-4,-5,-6,-7\right).$$

In [ ]:
f3 = np.copy(f_X)
f3[4] = -f_X[4]
f3[5] = -f_X[5]
f3[6] = -f_X[6]
f3[7] = -f_X[7]

In [ ]:
p4_gate = load_array(p_X)
f4_gate = load_array(f3,id_name = "2")

In [ ]:
oracle4 = qlm.QRoutine()
register4 = oracle4.new_wires(n+2)
oracle4.apply(uniform_distribution(n),register4[:n])
oracle4.apply(p4_gate, [register4[:n],register4[n]])
oracle4.apply(f4_gate, [register4[:n],register4[n+1]])
oracle4.apply(uniform_distribution(n),register4[:n])

In [ ]:
target = [0,0,0,0,0]
index = [0,1,2,3,4]
iqae = IQAE(oracle = oracle4,target = target, index = index)

In [ ]:
epsilon = 0.001
N = 556
alpha = 0.05 
bounds = iqae.run(epsilon = epsilon,N = N,alpha = alpha)

In [ ]:
print("Estimation: ",2**n*np.sqrt(bounds[0]),2**n*np.sqrt(bounds[1]))
print("Exact solution: ",np.dot(p_X,f3))

Here we see that the module is correct but not the sign. Of course this becomes critical as it is not the same obtining an expected positve result or a negative one. For instance, think of the return on an investment.

In [ ]:
iqae.display_information(epsilon = epsilon,N = N,alpha = alpha)

## 6. A more definitive approach

The solution to this is using an algorithm which distinguishes the sign of the underlying amplitude. For that we propose the RQAE, for more information check the corresponding notebook and article.

In [ ]:
from QQuantLib.AE.real_quantum_ae import RQAE

In [ ]:
target = [0]*(n+2)
index = np.arange(n+2)
rqae = RQAE(oracle = oracle4,target = target, index = index)

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
bounds = rqae.run(q = q, epsilon = epsilon,gamma = gamma)

In [ ]:
print("Estimation: ",2**n*bounds[0],2**n*bounds[1])
print("Exact solution: ",np.dot(p_X,f3))

In [ ]:
rqae.display_information(q = q, epsilon = epsilon,gamma = gamma)

## 7. Call option under the Black-Scholes model

Now we will use this same strategy to compute the prices of a plain vanilla under the assumptions of the Black-Scholes model. In order to do that, we have to choose some parameters for the pricing model, in this case:
- Current underlying price ($S_0$): 2
- Risk-free rate ($r$): 4\%
- The volatility ($\sigma$): 10\%

Next, we define the parameters of the call option:
- Maturity ($T$): 300 days 
- Strike ($K$): 1.9

In [ ]:
S0 = 2
r = 0.04
sigma = 0.1
T = 300/365
K = 1.9

The price of a call option can be approximated as:
$$C(S_0,K,T) = e^{-rT}\mathbb{E}_r[(S_T-K)^+]$$
In the next cell we compute the approximated value when we approximate the probability distribution and the payoff with $2^n$ points.

In [ ]:
from QQuantLib.utils.utils import bs_probability, call_payoff

In [ ]:
n = 3
x = np.linspace(1,3,2**n)
probability = bs_probability(x,S0,r,sigma,T)
payoff = call_payoff(x,K)
classical_price_estimation = np.exp(-r*T)*np.dot(probability,payoff)

Now we will compute the approximated price via the quantum computer. Before we load the probability and the payoff into the quantum computer we have to normalise the payoff function.

In [ ]:
payoff_normalisation = np.max(payoff)
payoff_normalised = payoff/payoff_normalisation

In [ ]:
p5_gate = load_array(probability,id_name = "probability")
f5_gate = load_array(payoff_normalised,id_name = "payoff")

In [ ]:
oracle5 = qlm.QRoutine()
register5 = oracle5.new_wires(n+2)
oracle5.apply(uniform_distribution(n),register5[:n])
oracle5.apply(p5_gate, [register5[:n],register5[n]])
oracle5.apply(f5_gate, [register5[:n],register5[n+1]])
oracle5.apply(uniform_distribution(n),register5[:n])

In [ ]:
target = [0]*(n+2)
index = np.arange(n+2)
rqae = RQAE(oracle = oracle5,target = target, index = index)

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
bounds_rqae = rqae.run(q = q, epsilon = epsilon,gamma = gamma)

In [ ]:
quantum_price_estimation = (bounds_rqae[0]+bounds_rqae[1])/2*2**n*payoff_normalisation*np.exp(-r*T)

Last, we use function *bs_call_price* to compute the exact price and compare it with the classical and quantum estimation

In [ ]:
from QQuantLib.utils.utils import bs_call_price

In [ ]:
exact_price = bs_call_price(S0,r,sigma,T,K)

In [ ]:
print("Exact price: ",exact_price)
print("Classical estimation: ",classical_price_estimation)
print("Quantum estimation: ",quantum_price_estimation)

Note that this procedure does not provide any quantum advantage.

## 8. Futures payoff under Black-Scholes model 

To emphasize the advantages of the RQAE method we will use the same strategy to compute the prices of a derivatives contract with linear payoff under the assumptions of the Black-Scholes model. In order to do that, we have to choose some parameters for the pricing model, in this case:
- Current underlying price ($S_0$): 2
- Risk-free rate ($r$): 4\%
- The volatility ($\sigma$): 10\%

Next, we define the parameters of the derivatives contract:
- Maturity ($T$): 300 days 
- Strike ($K$): 2.5

In [ ]:
S0 = 2
r = 0.04
sigma = 0.1
T = 300/365
K = 2.5

The price of this derivatives contract with linear payoff can be approximated:
$$C(S_0,K,T) = e^{-rT}\mathbb{E}_r[S_T-K]$$
In the next cell we compute the approximated value when we approximate the probability distribution and the payoff with $2^n$ points.

In [ ]:
n = 3
x = np.linspace(1,3,2**n)
probability = bs_probability(x,S0,r,sigma,T)
payoff = x-K
classical_price_estimation = np.exp(-r*T)*np.dot(probability,payoff)
print("Classical price estimation: ",classical_price_estimation)

As we see the price of this contract is negative. As we saw before, the standard quantum procedure won't work so we use RQAE. With it we compute the approximated price via the quantum computer. Before we load the probability and the payoff into the quantum computer we have to normalise the payoff function.

In [ ]:
payoff_normalisation = np.max(np.abs(payoff))
payoff_normalised = payoff/payoff_normalisation

In [ ]:
p5_gate = load_array(probability,id_name = "probability")
f5_gate = load_array(payoff_normalised,id_name = "payoff")

In [ ]:
oracle5 = qlm.QRoutine()
register5 = oracle5.new_wires(n+2)
oracle5.apply(uniform_distribution(n),register5[:n])
oracle5.apply(p5_gate, [register5[:n],register5[n]])
oracle5.apply(f5_gate, [register5[:n],register5[n+1]])
oracle5.apply(uniform_distribution(n),register5[:n])

In [ ]:
target = [0]*(n+2)
index = np.arange(n+2)
rqae = RQAE(oracle = oracle5,target = target, index = index)

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
bounds_rqae = rqae.run(q = q, epsilon = epsilon,gamma = gamma)

In [ ]:
quantum_price_estimation = (bounds_rqae[0]+bounds_rqae[1])/2*2**n*payoff_normalisation*np.exp(-r*T)

Last, we compare the classical and quantum estimation:

In [ ]:
print("Classical estimation: ",classical_price_estimation)
print("Quantum estimation: ",quantum_price_estimation)

## 8. Computation of VaR

For a given confidence level $\alpha$, $VaR_\alpha(X)$ is the smallest value $x$ such that 
$$P[X\leq x]\geq (1-\alpha).$$ To compute this quantity we can do a binary search.

### 8.1 Classical binary search

A binary search for $N = 2^n$ discrete probabilities works as follows:
- We begin by computing the cumulative sum for the first $2^{n-1}$ probabilities.
- If the probability is lower than the one we demand, we set $2^{n-1} = 2^{n-1}+2^{n-2}$ entries. Otherwise, we set $2^{n-1} = 2^{n-1}-2^{n-2}$ entries.
- Now we can start in the first step until $N = 1$, then we stop.
In the following cells we do this process in an iterative fashion to compute the VaR

We will do this process with the probability distribution from Black-Scholes equation

In [ ]:
probability = bs_probability(x,S0,r,sigma,T)
#probability = np.array([0,0,0,0,0.96,0.04,0,0])

In [ ]:
alpha = 0.05
index = 0
sign = 1
for i in range(1,n+1):
    index = index+sign*2**(n-i)
    suma = np.sum(probability[0:index])
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)

In [ ]:
np.sum(probability[:index])

### 8.2 Quantum binary search with IQAE

The quantum binary search using IQAE proceeds in the same manner as the classical binary search. The main difference is in the quanutm way we perform the cumulative sum. For that purpose we have defined the step function $s_i^N$. This function flips the leftmost qubit of all the states $j$ with the condition $j\geq i$. 

Let us give an example with $3+1$ qubits. We start with the state:
$$|\phi\rangle_0 = |0\rangle\otimes\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle+\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right].$$
An application of the function $s_4^8$ yields:
$$s_4^8|\phi\rangle_0 = |0\rangle\otimes\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle\right]+|1\rangle\otimes\left[\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right].$$
Now, performing the sum of the first four entries is equivalent to measuring the probability of obtaining $|0\rangle$ in the leftmost qubit. This is the strategy to perform the cumulative sums.

Note that, in the cell where we perform the binary search we only substitute the line of the cumulative sum with this new strategy, the rest remains the same.


In [ ]:
from QQuantLib.DL.data_loading import step_array

In [ ]:
probability = bs_probability(x,S0,r,sigma,T)
#probability = np.array([0,0,0,0,0.96,0.04,0,0])
p_gate = load_probability(probability)

In [ ]:
N = 100
epsilon = 0.01

alpha = 0.05
index = 0
sign = 1
for i in range(1,n+1):
    index = index+sign*2**(n-i)
    # Sum process done in the quantum computer
    routine = qlm.QRoutine()
    register = routine.new_wires(n+1)
    routine.apply(p_gate,register[:n])
    routine.apply(step_array(index,2**n),register)
    ##########################################
    iqae = IQAE(oracle = routine,target = [0], index = [n])
    bounds = iqae.run(epsilon = epsilon,N = N,alpha = 0.05)
    suma = (bounds[0]+bounds[1])/2
    ##########################################
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)

In [ ]:
np.sum(probability[:index])

### 8.3 Quantum binary search with RQAE

Once again, the quantum binary search with RQAE proceeds in the same manner as the classical binary search. The main difference is in the quanutm way we perform the cumulative sum. As here we work with the amplitudes instead of the probabilities we have to perform the sum in the quantum state.

Let us give an example with $3+1$ qubits. We start with the state:
$$|\phi\rangle_0 = \dfrac{1}{\sqrt{2^3}}|0\rangle\otimes |0\rangle\otimes\left[p_0|0\rangle+p_1|1\rangle+p_2|2\rangle+p_3|3\rangle+p_4|4\rangle+p_5|5\rangle+p_6|6\rangle+p_7|7\rangle\right]+ ....$$
An application of the step function $s_4^8$ yields:
$$s_4^8|\phi\rangle_0 =  \dfrac{1}{\sqrt{2^3}}|0\rangle\otimes|0\rangle\otimes\left[p_0|0\rangle+p_1|1\rangle+p_2|2\rangle+p_3|3\rangle\right]+|1\rangle\otimes|0\rangle\otimes\left[p_4|4\rangle+p_5|5\rangle+p_6|6\rangle+p_7|7\rangle\right]+....$$
Now, performing the sum of the first four entries is equivalent to applying Hadamard gates to the first entries:
$$H^{\otimes3}|\phi\rangle_1 =  \dfrac{p_0+p_1+p_2+p_3}{2^3}|0\rangle\otimes|0\rangle\otimes|0\rangle+...$$
This is the strategy to perform the cumulative sums.

Note that, in the cell where we perform the binary search we only substitute the line of the cumulative sum with this new strategy, the rest remains the same.



In [ ]:
probability = bs_probability(x,S0,r,sigma,T)
#probability = np.array([0,0,0,0,0.96,0.04,0,0])
p_gate = load_array(probability,id_name = "probability")

In [ ]:
gamma = 0.05
q = 2
epsilon = 0.01

alpha = 0.05
index = 0
sign = 1
for i in range(1,n+1):
    index = index+sign*2**(n-i)
    # Sum process done in the quantum computer
    routine = qlm.QRoutine()
    register = routine.new_wires(n+2)
    routine.apply(uniform_distribution(n),register[:n])
    routine.apply(p_gate,[register[:n],register[n]])
    routine.apply(step_array(index,2**n),[register[0:n],register[n+1]])
    routine.apply(uniform_distribution(n),register[:n])
    ##########################################
    rqae = RQAE(oracle = routine,target = [0]*(n+2), index = np.arange(n+2))
    bounds = rqae.run(q = q,epsilon = epsilon,gamma = gamma)
    suma = np.abs(bounds[1]*2**n)
    ##########################################
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)

In [ ]:
np.sum(probability[:index])